1- Connecting with Google Drive

In [10]:
#Mounting the Google Drive

from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/ChatBot'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2 - Importing relevant libraries

In [11]:
#Importing Libraries

import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

3- Reading the JSON file

In [12]:
#Loading the Dataset: intents.json

data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

4- Identifying Feature and Target for the NLP Model

In [15]:
#Creating data_X and data_Y

words = [] #For Bow model/ vocabulary for patterns
classes = [] # For Bow model/vocabulary for tags
data_X = [] # For storing each pattern
data_y = [] # For storing tag corresponding to each pattern in data_x
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
      tokens = nltk.word_tokenize(pattern) # tokenize each pattern
      words.extend(tokens) #append tokens to words
      data_X.append(pattern) #appending pattern to data_X
      data_y.append(intent["tag"]) , #appending the associated tag to each pattern

    # adding the tag to the classes if it's not there already

    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# intilialize lemmatizer to get stem of words

lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in the punctuation

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

#sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur

words = sorted(set(words))
classes = sorted(set(classes))

5- Making the data Machine-friendly

In [18]:
# Text to Numbers

training = []
out_empty = [0] * len(classes)

# creating the bag of words model

for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)

    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1

    #add the one hot encoded BowW and associated classes to training
    training.append([bow, output_row])

# shuffle the data and convert it to an array

random.shuffle(training)
training = np.array(training, dtype=object)

#split the features and target labels

train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))


6- Building the Neural Network Model

In [19]:

#The Neural Network Model

model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           7,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │           1,040 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,848 (65.81 KB)

 Trainable params: 16,848 (65.81 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.0592 - loss: 2.7965
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1067 - loss: 2.6845 
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1555 - loss: 2.4782
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3260 - loss: 2.4246
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3747 - loss: 2.2108
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3793 - loss: 2.2442
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3793 - loss: 1.9563
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4327 - loss: 2.0082
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4444 - loss: 1.9323
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5777 - loss: 1.5175
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6845 - loss: 1.4038 
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.604

7- Pre-processing the User's Input

In [23]:
#Preprocessing inputs

def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

''' Clean_text(text): This function receives text (string) as an input and then
    tokenizes it using the nltk.word_tokenize(). Each token is then converted into
    its root form using a lemmatizer. The output is basically a list of words in their root form.
'''
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len (vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:
                bow[idx] = 1
    return np.array(bow)

''' Bag_of_words(text, vocab): This function calls the above function,
    converts the text into an array using the bag-of-words model using the input
    vocabulary, and then returns the same array.
'''

def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0] #Extracting probabilities
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True) #sorting by values of probability in decreasing order
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability
    return return_list

''' Pred_class(text, vocab, labels): This function takes text, vocab,
    and labels as input and returns a list that contains a tag corresponding
    to the highest probability.
'''
def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = "Sorry! I don't understand."
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"] == tag:
                result = random.choice(i["responses"])
                break
    return result

''' Get_response(intents_list, intents_json): This function takes in the tag
    returned by the previous function and uses it to randomly choose a response
    corresponding to the same tag in intent.json. And, if the intents_list is
    empty, that is when the probability does not cross the threshold, we will
    pass the string “Sorry! I don’t understand” as ChatBot’s response.
'''

' Get_response(intents_list, intents_json): This function takes in the tag\n    returned by the previous function and uses it to randomly choose a response\n    corresponding to the same tag in intent.json. And, if the intents_list is\n    empty, that is when the probability does not cross the threshold, we will\n    pass the string “Sorry! I don’t understand” as ChatBot’s response.\n'

8- Calling relevant functions and interacting with ChatBot

In [24]:
# Interacting with Chatbot

print("Press 0 if you don't want to chat with our ChatBot")
while True:
    message = input("")
    if message == "0":
        break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Press 0 if you don't want to chat with our ChatBot
Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
That is perfect!
I said hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Hey!
How are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Sorry! I don't understand.
How are you doing?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Actually, I'm chatting with somebody
Pretty impressive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
That is perfect!
How are your predictions?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Sorry! I don't understand.
Well I think you need to train
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Sorry! I don't understand.
ok
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
That is perfect!
Bye!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
So, everything's okay!
0
